In [75]:
%matplotlib inline

In [294]:
import pandas as pd
import subprocess
import re
import json
from datetime import date

In [351]:
events = [{'hc': 'Tu BiShvat', 'short': 'tb'},
          {'hc': 'Erev Purim', 'short': 'pu'},
          {'hc': 'Erev Pesach', 'short': 'pe'},
          {'hc': 'Erev Shavuot', 'short': 'sh'},
          {'hc': 'Erev Tish\'a B\'Av', 'short': 'ta'},
          {'hc': 'Erev Rosh Hashana', 'short': 'r'},
          {'hc': 'Erev Yom Kippur', 'short': 'y'},
          {'hc': 'Erev Sukkot', 'short': 'su'},
          {'hc': 'Chanukah: 1 Candle', 'short': 'c'}]

In [358]:
def get_dates(year):
    hebcal_str = subprocess.run('../hebcal-master/./hebcal ' + str(year), shell=True, stdout=subprocess.PIPE, encoding='utf8').stdout
    results = {}
    heb_year = int(re.search(r'Rosh Hashana ([0-9]{4})', hebcal_str).group(1))
    leaps = [3, 6, 8, 11, 14, 17, 0]
    rh_date = None

    for i, event in enumerate(events):
        date_search = re.search(r'(([0-9]{1,2})/([0-9]{1,2})/[0-9]{4}) ' + event['hc'], hebcal_str)
        month = int(date_search.group(2))
        day = int(date_search.group(3))
        results[event['short']] = {'e': event['short'], 'y': year, 'm': month, 'd': day}
        if event['hc'] == 'Erev Rosh Hashana': rh_date = date(year, month, day)
    
    for event_name in results:
        this_event = results[event_name]
        is_prev = date(this_event['y'], this_event['m'], this_event['d']) < rh_date
        this_heb_year = heb_year - (1 if is_prev else 0)
        this_event['hy'] = this_heb_year
        this_event['l'] = (this_heb_year % 19) in leaps
    return results.values()

In [361]:
dates = []
for y in range(1583, 2400):
    dates.extend(get_dates(y))

In [347]:
pd.DataFrame(dates).query('leap == False & event == "Erev Pesach" & month == 4 & day > 19')

,day,event,heb_year,leap,month,year


In [362]:
# export raw data
with open('data.json', 'w') as file:
    json.dump(dates, file)

In [268]:
# export aggregated data
pd.DataFrame(pd.DataFrame(dates).groupby(['event', 'month', 'day']).size(), columns=['count']).reset_index().to_json('agg_data.json', orient='records')

event    Tu BiShvat
month            12
day              31
count            17
dtype: object